In [4]:
import numpy as np
import pandas as pd
from typing import Any, Dict, List, Optional, Sequence, Tuple

from __future__ import annotations

import os
import re
import sys
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Sequence, Tuple

import numpy as np
import pyxdf
from scipy.io import savemat
from scipy.stats import zscore

def stream_name(stream: Dict[str, Any]) -> str:
    return stream.get("info", {}).get("name", [""])[0]

def stream_type(stream: Dict[str, Any]) -> str:
    return stream.get("info", {}).get("type", [""])[0]

def get_channel_labels(stream: Optional[Dict[str, Any]]) -> Optional[List[str]]:
    if stream is None:
        return None
    try:
        chs = stream["info"]["desc"][0]["channels"][0]["channel"]
        return [ch.get("label", [""])[0] for ch in chs]
    except Exception:
        return None


def pick_stream(
    streams: Sequence[Dict[str, Any]],
    force_name: Optional[str],
    kind: str,  # "markers" | "eeg" | "fnirs"
) -> Optional[Dict[str, Any]]:
    """
    Select a stream either by exact name, or by heuristic using stream type/name.
    """
    if force_name:
        for s in streams:
            if stream_name(s) == force_name:
                return s
        raise RuntimeError(f"Could not find {kind} stream with name='{force_name}'")

    for s in streams:
        name = stream_name(s)
        stype = stream_type(s)
        name_l = name.lower()
        stype_u = stype.upper()

        if kind == "markers":
            if stype_u == "MARKERS" or "marker" in name_l:
                return s

        if kind == "eeg":
            if stype_u == "EEG" or re.search(r"\beeg\b", name_l):
                return s

        if kind == "fnirs":
            if stype_u in {"NIRS", "FNIRS"} or re.search(r"(fnirs|nirs|nirstar)", name_l):
                return s

    return None

def print_streams(streams: Sequence[Dict[str, Any]]) -> None:
    print("\nStreams found:")
    for i, s in enumerate(streams):
        print(
            f"  [{i}] name='{stream_name(s)}' type='{stream_type(s)}' "
            f"samples={len(s.get('time_stamps', []))}"
        )
# =============================================================================
# Marker parsing
# =============================================================================

def flatten_marker_labels(markers_time_series: Sequence[Any]) -> List[str]:
    """
    LabRecorder markers often look like: [['label'], ['label2'], ...]
    """
    if not markers_time_series:
        return []
    first = markers_time_series[0]
    if isinstance(first, list):
        return [m[0] if m else "" for m in markers_time_series]
    return [str(m) for m in markers_time_series]


@dataclass(frozen=True)
class StartEvent:
    t0: float
    label: str
    user_id: Optional[int]
    attempt: Optional[int]

def extract_start_events(
    markers_stream: Dict[str, Any],
    user_id_filter: Optional[int],
    keep_first_baseline: bool,
) -> List[StartEvent]:
    """
    Keeps:
    - ALL RollerCoasterStarted markers
    - optionally ONLY the FIRST RollerCoasterBaselineStarted with attempt=0 per user

    TODO : add final baseline for this as well
    """
    markers_times = np.asarray(markers_stream["time_stamps"])
    markers_labels = flatten_marker_labels(markers_stream["time_series"])

    rc_start_pat = re.compile(r"^RollerCoasterStarted\b", re.IGNORECASE)
    baseline_start_pat = re.compile(r"^RollerCoasterBaselineStarted\b", re.IGNORECASE)
    attempt_pat = re.compile(r"\battempt=(\d+)\b", re.IGNORECASE)
    user_pat = re.compile(r"\buser=(\d+)\b", re.IGNORECASE)

    kept_first_baseline_users = set()
    events: List[StartEvent] = []

    for t, lab in zip(markers_times, markers_labels):
        if not isinstance(lab, str):
            continue

        um = user_pat.search(lab)
        uid = int(um.group(1)) if um else None

        if user_id_filter is not None and uid != user_id_filter:
            continue

        am = attempt_pat.search(lab)
        attempt = int(am.group(1)) if am else None

        if rc_start_pat.search(lab):
            events.append(StartEvent(float(t), lab, uid, attempt))
            continue

        if keep_first_baseline and baseline_start_pat.search(lab) and attempt == 0:
            if uid not in kept_first_baseline_users:
                events.append(StartEvent(float(t), lab, uid, attempt))
                kept_first_baseline_users.add(uid)

    events.sort(key=lambda e: e.t0)
    return events

In [10]:
xdf_path = "./Data/Marker_Check/sub-000_ses-S001_task-Default_run-001_eeg.xdf"
print(f"Loading XDF: {xdf_path}")
streams, _header = pyxdf.load_xdf(xdf_path)
print_streams(streams)

markers_stream = pick_stream(streams, "Game_Markers", "markers")

Loading XDF: ./Data/Marker_Check/sub-000_ses-S001_task-Default_run-001_eeg.xdf

Streams found:
  [0] name='Game_Markers' type='Markers' samples=32
  [1] name='FMS_Score' type='Survey' samples=48
  [2] name='Coaster' type='Object' samples=26925
  [3] name='HMD_MotionData' type='VR' samples=26924


In [11]:
markers_stream['time_series']

[['Begining_Eyes_closed|session=RollerCoasterVR|user=00|attempt=0'],
 ['SurveySubmitted|session=RollerCoasterVR|user=00|attempt=0'],
 ['RollerCoasterBaselineStarted|session=RollerCoasterVR|user=00|attempt=0'],
 ['RollerCoasterBaselineFinished|session=RollerCoasterVR|user=00|attempt=0'],
 ['SurveySubmitted|session=RollerCoasterVR|user=00|attempt=0'],
 ['RollerCoasterStarted|session=RollerCoasterVR|user=00|attempt=1'],
 ['RollerCoasterFinished|session=RollerCoasterVR|user=00|attempt=1'],
 ['SurveySubmitted|session=RollerCoasterVR|user=00|attempt=1'],
 ['RollerCoasterStarted|session=RollerCoasterVR|user=00|attempt=2'],
 ['RollerCoasterFinished|session=RollerCoasterVR|user=00|attempt=2'],
 ['SurveySubmitted|session=RollerCoasterVR|user=00|attempt=2'],
 ['RollerCoasterStarted|session=RollerCoasterVR|user=00|attempt=3'],
 ['RollerCoasterFinished|session=RollerCoasterVR|user=00|attempt=3'],
 ['SurveySubmitted|session=RollerCoasterVR|user=00|attempt=3'],
 ['RollerCoasterStarted|session=RollerC

In [ ]:
[['Begining_Eyes_closed|session=RollerCoasterVR|user=00|attempt=0'],
 ['SurveySubmitted|session=RollerCoasterVR|user=00|attempt=0'],
 ['RollerCoasterBaselineStarted|session=RollerCoasterVR|user=00|attempt=0'],
 ['RollerCoasterBaselineFinished|session=RollerCoasterVR|user=00|attempt=0'],
 ['SurveySubmitted|session=RollerCoasterVR|user=00|attempt=0'],
 ['RollerCoasterStarted|session=RollerCoasterVR|user=00|attempt=1'],
 ['SurveyStarted|session=RollerCoasterVR|user=00|attempt=1'],
 ['RollerCoasterFinished|session=RollerCoasterVR|user=00|attempt=1'],
 ['SurveySubmitted|session=RollerCoasterVR|user=00|attempt=1'],
 ['RollerCoasterStarted|session=RollerCoasterVR|user=00|attempt=2'],
 ['SurveyStarted|session=RollerCoasterVR|user=00|attempt=2'],
 ['RollerCoasterFinished|session=RollerCoasterVR|user=00|attempt=2'],
 ['SurveySubmitted|session=RollerCoasterVR|user=00|attempt=2'],
 ['RollerCoasterStarted|session=RollerCoasterVR|user=00|attempt=3'],
 ['SurveyStarted|session=RollerCoasterVR|user=00|attempt=3'],
 ['RollerCoasterFinished|session=RollerCoasterVR|user=00|attempt=3'],
 ['SurveySubmitted|session=RollerCoasterVR|user=00|attempt=3'],
 ['RollerCoasterStarted|session=RollerCoasterVR|user=00|attempt=4'],
 ['SurveyStarted|session=RollerCoasterVR|user=00|attempt=4'],
 ['RollerCoasterFinished|session=RollerCoasterVR|user=00|attempt=4'],
 ['SurveySubmitted|session=RollerCoasterVR|user=00|attempt=4'],
 ['RollerCoasterStarted|session=RollerCoasterVR|user=00|attempt=5'],
 ['SurveyStarted|session=RollerCoasterVR|user=00|attempt=5'],
 ['RollerCoasterFinished|session=RollerCoasterVR|user=00|attempt=5'],
 ['SurveySubmitted|session=RollerCoasterVR|user=00|attempt=5'],
 ['RollerCoasterStarted|session=RollerCoasterVR|user=00|attempt=6'],
 ['SurveyStarted|session=RollerCoasterVR|user=00|attempt=6'],
 ['RollerCoasterFinished|session=RollerCoasterVR|user=00|attempt=6'],
 ['SurveySubmitted|session=RollerCoasterVR|user=00|attempt=6'],
 ['SurveySubmitted|session=RollerCoasterVR|user=00|attempt=6'],
 ['Extra_Round_started|session=RollerCoasterVR|user=00|attempt=6'],
 ['RollerCoasterStarted|session=RollerCoasterVR|user=00|attempt=6'],
 ['SurveyStarted|session=RollerCoasterVR|user=00|attempt=6'],
 ['RollerCoasterFinished|session=RollerCoasterVR|user=00|attempt=6'],
 ['Extra_Round_finished|session=RollerCoasterVR|user=00|attempt=6'],
 ['SurveySubmitted|session=RollerCoasterVR|user=00|attempt=6'],
 ['End_Eyes_closed|session=RollerCoasterVR|user=00|attempt=6'],
 ['Debrief_Started|session=RollerCoasterVR|user=00|attempt=6']]